In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.utils.data import DataLoader, TensorDataset, Subset
import sklearn.metrics as sk

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
data = np.load("/content/drive/MyDrive/Unilever Datasets, Sliding Window 13/HEALTHY SNACKING_dataset_processed (1).npz")

In [4]:

# Extract the arrays
X = data['X']  # Shape: (timesteps, input_sequence_length, nodes, features)
Y = data['Y']  # Shape: (timesteps, output_sequence_length, nodes, features)

# Drop all features except the first one for both X and Y
X = X[:, :, :, :1]  # Keep only the first feature
Y = Y[:, :, :, :1]  # Keep only the first feature


In [5]:
# Reshape Y to match the output format, if necessary
# Y is expected to have shape (timestep/index, prediction window, nodes, 1)
# If Y's shape is already correct, skip this step
X = X.squeeze(-1)
Y = Y.squeeze(-1)  # Remove the last dimension

In [14]:
print("Y shape after squeezing:", Y.shape)
print("Y sample:", Y[:5])  # Print a small portion of Y to verify


Y shape after squeezing: (167, 13, 1082)
Y sample: [[[360. 360.   0. ...   0.   0.   0.]
  [360. 360.   2. ...   0.   0.   0.]
  [120. 120.   0. ...   0.   0.   0.]
  ...
  [  0. 240.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]]

 [[360. 360.   2. ...   0.   0.   0.]
  [120. 120.   0. ...   0.   0.   0.]
  [  0. 480.   0. ...   0.   0.   0.]
  ...
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]]

 [[120. 120.   0. ...   0.   0.   0.]
  [  0. 480.   0. ...   0.   0.   0.]
  [360.   0.   0. ...   0.   0.   0.]
  ...
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]]

 [[  0. 480.   0. ...   0.   0.   0.]
  [360.   0.   0. ...   0.   0.   0.]
  [  0. 600.   0. ...   0.   0.   0.]
  ...
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   3. ...   0.   0.   0.]]

 [[36

In [7]:
# Verify that the input and output sequence lengths are 13
assert X.shape[1] == 13, f"Expected input sequence length of 13, got {X.shape[1]}"
assert Y.shape[1] == 13, f"Expected output sequence length of 13, got {Y.shape[1]}"

# Split the data into train, validation, and test sets (70%, 15%, 15%)
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)
test_size = len(X) - train_size - val_size

X_train, X_val, X_test = X[:train_size], X[train_size:train_size + val_size], X[train_size + val_size:]
y_train, y_val, y_test = Y[:train_size], Y[train_size:train_size + val_size], Y[train_size + val_size:]

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Print shapes to verify
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: torch.Size([116, 13, 1082])
y_train shape: torch.Size([116, 13, 1082])
X_val shape: torch.Size([25, 13, 1082])
y_val shape: torch.Size([25, 13, 1082])
X_test shape: torch.Size([26, 13, 1082])
y_test shape: torch.Size([26, 13, 1082])


In [8]:
# Define the GRU model with Attention
class AirModelWithAttention(nn.Module):
    def __init__(self, input_sz, hidden_sz, output_sz, n_steps, dropout_prob=0.1):
        super().__init__()
        self.hidden_sz = hidden_sz
        self.gru = nn.GRU(
            input_size=input_sz,
            hidden_size=hidden_sz,
            num_layers=2,
            batch_first=True,
            bidirectional=False
        )
        self.dropout = nn.Dropout(p=dropout_prob)

        # Attention layer
        self.attention = nn.Linear(hidden_sz, 1)

        # Output layer
        self.linear = nn.Linear(hidden_sz, output_sz * n_steps)
        self.output_sz = output_sz
        self.n_steps = n_steps

    def forward(self, x):
        # x: [batch_size, seq_length, input_size]
        gru_output, _ = self.gru(x)  # gru_output: [batch_size, seq_length, hidden_sz]

        # Apply attention
        # Compute attention scores
        attn_scores = self.attention(gru_output)  # attn_scores: [batch_size, seq_length, 1]
        attn_scores = attn_scores.squeeze(-1)     # attn_scores: [batch_size, seq_length]

        # Normalize attention scores to get attention weights
        attn_weights = torch.softmax(attn_scores, dim=1)  # attn_weights: [batch_size, seq_length]

        # Compute context vector as weighted sum of GRU outputs
        context_vector = torch.sum(gru_output * attn_weights.unsqueeze(-1), dim=1)  # context_vector: [batch_size, hidden_sz]

        # Apply dropout
        context_vector = self.dropout(context_vector)

        # Pass context vector through the output layer
        output = self.linear(context_vector)
        output = torch.relu(output)

        # Reshape output to match the desired format
        output = output.view(-1, self.n_steps, self.output_sz)
        return output


In [9]:
# Model parameters
input_size = X_train.shape[2]
output_size = y_train.shape[2]
model = AirModelWithAttention(
    input_sz=input_size,
    hidden_sz=50,
    output_sz=output_size,
    n_steps=13,  # Updated to match the forecasting window
    dropout_prob=0.1  # Added dropout for regularization
)

# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Reduced learning rate for stability
loss_fn = nn.MSELoss()

# Prepare the data loader
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=16, shuffle=False)

# Lists to store metrics
train_metrics = {'RMSE': [], 'MAE': [], 'R2': [], 'MSE': [], 'MAPE': [], 'WMAPE': []}
val_metrics = {'RMSE': [], 'MAE': [], 'R2': [], 'MSE': [], 'MAPE': [], 'WMAPE': []}
test_metrics = {'RMSE': [], 'MAE': [], 'R2': [], 'MSE': [], 'MAPE': [], 'WMAPE': []}

# Function to calculate WMAPE
def wmape(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

# Function to calculate MAPE assuming there are y_actuals that are equal to 0
# def mape(y_true, y_pred):
#     y_true = y_true.flatten()
#     y_pred = y_pred.flatten()
#     # Create a mask for non-zero actuals
#     mask = y_true != 0
#     # Ensure there are non-zero actuals to prevent division by zero in the mean
#     if np.sum(mask) == 0:
#         return np.nan  # Or handle appropriately
#     # Compute MAPE only on non-zero actuals
#     return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100


def mape(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100  # Added 1e-8 to avoid division by zero

# Training loop
n_epochs = 500  # Reduced epochs for faster training during testing
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation every epoch
    model.eval()
    with torch.no_grad():
        # Helper function to calculate metrics
        def calculate_metrics(y_true, y_pred, n):
            y_true = y_true[:, :n, :].cpu().numpy()
            y_pred = y_pred[:, :n, :].cpu().numpy()

            mse = mean_squared_error(y_true.flatten(), y_pred.flatten())
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_true.flatten(), y_pred.flatten())
            r2 = r2_score(y_true.flatten(), y_pred.flatten())
            mape_value = mape(y_true, y_pred)
            wmape_value = wmape(y_true, y_pred) * 100  # Expressed as percentage
            return mse, rmse, mae, r2, mape_value, wmape_value

        # Training metrics
        y_pred_train = model(X_train)
        mse_train, rmse_train, mae_train, r2_train, mape_train, wmape_train = calculate_metrics(y_train, y_pred_train, 13)
        train_metrics['MSE'].append(mse_train)
        train_metrics['RMSE'].append(rmse_train)
        train_metrics['MAE'].append(mae_train)
        train_metrics['R2'].append(r2_train)
        train_metrics['MAPE'].append(mape_train)
        train_metrics['WMAPE'].append(wmape_train)

        # Validation metrics
        y_pred_val = model(X_val)
        mse_val, rmse_val, mae_val, r2_val, mape_val, wmape_val = calculate_metrics(y_val, y_pred_val, 13)
        val_metrics['MSE'].append(mse_val)
        val_metrics['RMSE'].append(rmse_val)
        val_metrics['MAE'].append(mae_val)
        val_metrics['R2'].append(r2_val)
        val_metrics['MAPE'].append(mape_val)
        val_metrics['WMAPE'].append(wmape_val)

        # Testing metrics
        y_pred_test = model(X_test)
        mse_test, rmse_test, mae_test, r2_test, mape_test, wmape_test = calculate_metrics(y_test, y_pred_test, 13)
        test_metrics['MSE'].append(mse_test)
        test_metrics['RMSE'].append(rmse_test)
        test_metrics['MAE'].append(mae_test)
        test_metrics['R2'].append(r2_test)
        test_metrics['MAPE'].append(mape_test)
        test_metrics['WMAPE'].append(wmape_test)

    # Print metrics every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}")
        print(f"Train - RMSE: {rmse_train:.4f}, MAE: {mae_train:.4f}, R²: {r2_train:.4f}, MAPE: {mape_train:.2f}%, WMAPE: {wmape_train:.2f}%")
        print(f"Val   - RMSE: {rmse_val:.4f}, MAE: {mae_val:.4f}, R²: {r2_val:.4f}, MAPE: {mape_val:.2f}%, WMAPE: {wmape_val:.2f}%")
        print(f"Test  - RMSE: {rmse_test:.4f}, MAE: {mae_test:.4f}, R²: {r2_test:.4f}, MAPE: {mape_test:.2f}%, WMAPE: {wmape_test:.2f}%")
        print("-" * 80)

Epoch 10/500
Train - RMSE: 1282.7178, MAE: 206.7598, R²: -0.0257, MAPE: 6720681600.00%, WMAPE: 100.14%
Val   - RMSE: 1404.7418, MAE: 196.9530, R²: -0.0193, MAPE: 6915850400.00%, WMAPE: 100.16%
Test  - RMSE: 870.9332, MAE: 92.2183, R²: -0.0104, MAPE: 8758422400.00%, WMAPE: 100.77%
--------------------------------------------------------------------------------
Epoch 20/500
Train - RMSE: 1282.1152, MAE: 206.9928, R²: -0.0247, MAPE: 13036397600.00%, WMAPE: 100.25%
Val   - RMSE: 1404.2572, MAE: 197.2117, R²: -0.0186, MAPE: 13367412000.00%, WMAPE: 100.29%
Test  - RMSE: 870.5742, MAE: 92.9066, R²: -0.0095, MAPE: 17414747200.00%, WMAPE: 101.52%
--------------------------------------------------------------------------------
Epoch 30/500
Train - RMSE: 1281.5521, MAE: 207.1897, R²: -0.0238, MAPE: 18655955200.00%, WMAPE: 100.34%
Val   - RMSE: 1403.8041, MAE: 197.4288, R²: -0.0179, MAPE: 19097433600.00%, WMAPE: 100.40%
Test  - RMSE: 870.2415, MAE: 93.5151, R²: -0.0088, MAPE: 25125977600.00%, WMAP

In [10]:
# Calculate metrics for t=1 to t=5 and t=1 to t=13
def calculate_final_metrics(y_true, y_pred, n_steps_list):
    results = {}
    for n in n_steps_list:
        mse, rmse, mae, r2, mape_value, wmape_value = calculate_metrics(y_true, y_pred, n)
        results[f'n={n}'] = {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2,
            'MAPE': mape_value,
            'WMAPE': wmape_value
        }
    return results

# Evaluate on train set
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train)
    train_results = calculate_final_metrics(y_train, y_pred_train, [5, 13])

    y_pred_val = model(X_val)
    val_results = calculate_final_metrics(y_val, y_pred_val, [5, 13])

    y_pred_test = model(X_test)
    test_results = calculate_final_metrics(y_test, y_pred_test, [5, 13])

# Print final metrics
def print_results(results, dataset_name):
    print(f"\n{dataset_name} Metrics:")
    for n, metrics in results.items():
        print(f"\nFrom t=1 to t={n.split('=')[1]}:")
        print(f"MSE: {metrics['MSE']:.4f}")
        print(f"RMSE: {metrics['RMSE']:.4f}")
        print(f"MAE: {metrics['MAE']:.4f}")
        print(f"R²: {metrics['R2']:.4f}")
        print(f"MAPE: {metrics['MAPE']:.2f}%")
        print(f"WMAPE: {metrics['WMAPE']:.2f}%")

print_results(train_results, "Train")
print_results(val_results, "Validation")
print_results(test_results, "Test")


Train Metrics:

From t=1 to t=5:
MSE: 1583241.2500
RMSE: 1258.2692
MAE: 210.8065
R²: 0.0114
MAPE: 179570124800.00%
WMAPE: 102.17%

From t=1 to t=13:
MSE: 1585936.0000
RMSE: 1259.3395
MAE: 210.7849
R²: 0.0114
MAPE: 178545472000.00%
WMAPE: 102.09%

Validation Metrics:

From t=1 to t=5:
MSE: 1867829.0000
RMSE: 1366.6854
MAE: 203.5382
R²: 0.0078
MAPE: 182648678400.00%
WMAPE: 102.64%

From t=1 to t=13:
MSE: 1921527.8750
RMSE: 1386.1919
MAE: 202.2516
R²: 0.0074
MAPE: 184277222400.00%
WMAPE: 102.86%

Test Metrics:

From t=1 to t=5:
MSE: 795242.4375
RMSE: 891.7637
MAE: 110.4195
R²: 0.0161
MAPE: 259224140800.00%
WMAPE: 122.34%

From t=1 to t=13:
MSE: 737088.8125
RMSE: 858.5388
MAE: 111.5225
R²: 0.0182
MAPE: 258175564800.00%
WMAPE: 121.87%
